In [16]:
import tkinter as tk
from tkinter import messagebox
import random
import pandas as pd

# Cargar datos desde un archivo Excel
file_path = "preguntas.xlsx"
data = pd.read_excel(file_path)
data = data.astype(str)  # Convierte todos los valores en cadenas de texto

# Crear una instancia de Tk
window = tk.Tk()
window.title("Juego Trivia")

# Variables globales
question_index = 0
correct_answers = 0
answer_var = tk.StringVar()
responses = []  # Para almacenar las respuestas del estudiante
all_responses = []  # Para almacenar las respuestas de todos los estudiantes
student_name = ""  # Variable para almacenar el nombre del estudiante

# Función para mostrar la pregunta
def show_question():
    global question_index, correct_answers, student_name  # Asegúrate de que estas variables sean globales
    if question_index < len(data):
        # Obtener las opciones mezcladas
        options = [data["Respuesta Correcta"][question_index], data["R1"][question_index], data["R2"][question_index], data["R3"][question_index]]
        random.shuffle(options)

        # Actualizar el texto de la pregunta y las opciones
        question_label.config(text=data["Pregunta"][question_index])
        for i in range(4):
            radio_buttons[i].config(text=options[i], value=options[i])
        update_status()
    else:
        # Calcular el porcentaje de respuestas correctas
        percentage = (correct_answers / len(data)) * 100
        # Mostrar porcentaje de respuestas correctas
        messagebox.showinfo("Fin del juego", 
                            f"Juego terminado. Respondiste {correct_answers} preguntas correctas de {len(data)}.\n"
                            f"Porcentaje de respuestas correctas: {percentage:.2f}%")
        responses.append((student_name, correct_answers, len(data), percentage))
        all_responses.append(responses.copy())  # Guardar las respuestas del estudiante
        responses.clear()  # Limpiar las respuestas del estudiante actual
        question_index = 0  # Reiniciar el índice de preguntas
        correct_answers = 0
        student_name = ""
        name_frame.pack(pady=20)  # Mostrar el marco del nombre para un nuevo estudiante

# Función para actualizar el estado de la pregunta
def update_status():
    global question_index  # Asegúrate de que esta variable sea global
    status_label.config(text=f"Pregunta {question_index+1} de {len(data)}")

# Función para manejar la respuesta
def handle_answer():
    global question_index, correct_answers
    selected_answer = answer_var.get()
    if not selected_answer:
        messagebox.showwarning("Respuesta no seleccionada", "Por favor, selecciona una respuesta antes de continuar.")
        return
    
    correct_answer = data["Respuesta Correcta"][question_index]
    responses.append((data["Pregunta"][question_index], selected_answer, correct_answer))  # Guardar la respuesta del estudiante
    
    if selected_answer == correct_answer:
        correct_answers += 1
    else:
        messagebox.showerror("Respuesta Incorrecta", f"Incorrecto! La respuesta correcta era: {correct_answer}")
    
    question_index += 1  # Actualiza el índice para la próxima pregunta
    show_question()

# Función para guardar todas las respuestas en un archivo Excel
def save_responses_to_excel():
    if not all_responses:
        messagebox.showinfo("No hay datos", "No hay respuestas para guardar.")
        return

    df = pd.DataFrame(columns=['Nombre del Estudiante', 'Pregunta', 'Respuesta del Estudiante', 'Respuesta Correcta', 'Número de Respuestas Correctas', 'Número Total de Preguntas', 'Porcentaje de Respuestas Correctas'])
    
    for student_responses in all_responses:
        if not student_responses:
            continue
        
        student_name, num_correct, total_questions, percentage = student_responses[0]
        for question, student_answer, correct_answer in student_responses[1:]:
            df = df.append({
                'Nombre del Estudiante': student_name,
                'Pregunta': question,
                'Respuesta del Estudiante': student_answer,
                'Respuesta Correcta': correct_answer,
                'Número de Respuestas Correctas': num_correct,
                'Número Total de Preguntas': total_questions,
                'Porcentaje de Respuestas Correctas': percentage
            }, ignore_index=True)
    
    df.to_excel('respuestas_estudiantes.xlsx', index=False)

# Función para iniciar el juego
def start_game():
    global student_name
    student_name = name_entry.get()  # Capturar el nombre del estudiante
    if not student_name:
        messagebox.showwarning("Nombre Requerido", "Por favor, ingresa tu nombre antes de comenzar.")
    else:
        name_frame.pack_forget()  # Ocultar el marco del nombre
        question_frame.pack()     # Mostrar el marco de preguntas
        show_question()          # Mostrar la primera pregunta al iniciar el juego

# Función para salir del programa
def exit_program():
    save_responses_to_excel()  # Guardar todas las respuestas al salir
    window.destroy()

# Widgets para ingresar el nombre del estudiante
name_frame = tk.Frame(window)
name_label = tk.Label(name_frame, text="Ingresa tu nombre:")
name_label.pack(pady=(20, 5))
name_entry = tk.Entry(name_frame)
name_entry.pack(pady=5)
start_button = tk.Button(name_frame, text="Comenzar", command=start_game)
start_button.pack(pady=20)
name_frame.pack(pady=20)

# Widgets para las preguntas
question_frame = tk.Frame(window)
question_label = tk.Label(question_frame, text="", wraplength=400)
question_label.pack(pady=(20, 10))

radio_buttons = []
for _ in range(4):
    rb = tk.Radiobutton(question_frame, text="", variable=answer_var, wraplength=300)
    rb.pack(anchor="w")
    radio_buttons.append(rb)

answer_button = tk.Button(question_frame, text="Responder", command=handle_answer)
answer_button.pack(pady=20)

status_label = tk.Label(question_frame, text="")
status_label.pack()

# Botón para salir del programa
exit_button = tk.Button(window, text="Salir", command=exit_program)
exit_button.pack(pady=20)

# Iniciar el bucle principal de la interfaz gráfica
window.mainloop()



instalaciones necesarias para crear ejecutable exe:pip install pyinstaller

pip install python-docx
pip install pandas
pip install openpyxl

path para crear ejecutable ya el path de pyintaller no esta como varieble local, para corregir eso se deberá :

C:\Users\swild\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\Scripts\pyinstaller --onefile --windowed wild.py